In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit

In [3]:
df = pd.read_csv(fr"survey_lung_cancerdata.csv")
df['LUNG_CANCER'] = df['LUNG_CANCER'].replace({'YES': 1, 'NO': 0})
df['GENDER'] = df['GENDER'].replace({'M': 2, 'F': 1})
display(df[['SMOKING','LUNG_CANCER']])
df.corr()['LUNG_CANCER']

,SMOKING,LUNG_CANCER
0,1,1
1,2,1
2,1,0
3,2,0
4,1,0
...,...,...
304,1,1
305,2,1
306,2,1
307,2,1


GENDER                   0.067254
AGE                      0.089465
SMOKING                  0.058179
YELLOW_FINGERS           0.181339
ANXIETY                  0.144947
PEER_PRESSURE            0.186388
CHRONIC_DISEASE          0.110891
FATIGUE                  0.150673
ALLERGY                  0.327766
WHEEZING                 0.249300
ALCOHOL_CONSUMING        0.288533
COUGHING                 0.248570
SHORTNESS OF BREATH      0.060738
SWALLOWING_DIFFICULTY    0.259730
CHEST_PAIN               0.190451
LUNG_CANCER              1.000000
Name: LUNG_CANCER, dtype: float64

In [4]:
x = df[["AGE", 'PEER_PRESSURE', 'ALLERGY', 'ALCOHOL_CONSUMING', 'COUGHING','SWALLOWING_DIFFICULTY']]
y = df["LUNG_CANCER"]
sm.add_constant(x)
logit = Logit(y,x).fit()
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.272581
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:            LUNG_CANCER   No. Observations:                  309
Model:                          Logit   Df Residuals:                      303
Method:                           MLE   Df Model:                            5
Date:                Mon, 20 Nov 2023   Pseudo R-squ.:                  0.2810
Time:                        13:18:42   Log-Likelihood:                -84.228
converged:                       True   LL-Null:                       -117.15
Covariance Type:            nonrobust   LLR p-value:                 7.488e-13
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
AGE                      -0.0906      0.019     -4.792      0.000      -0.128      -0.

In [5]:
predikce = logit.predict(x)
bool_predikce = []

#>> zmena na true/false
#@ threshold
threshold = 0.75
for p in predikce:
    if p > threshold:
        p = True
    else:
        p = False
    bool_predikce.append(p)

#>> confusion matrix
tp = tn = fp = fn = 0
for i in range(0,len(predikce)):
    if bool_predikce[i] and y[i]:
        tp += 1
    elif not bool_predikce[i] and not y[i]:
        tn += 1
    elif bool_predikce[i] and not y[i]:
        fp += 1
    elif not bool_predikce[i] and y[i]:
        fn += 1
pd.DataFrame([[tp, fp],
                [tn, fn]],
                index = ["Actualy Positive", "Actualy Negative"], columns=["Predicted Positive", "Predicted Negative"])


,Predicted Positive,Predicted Negative
Actualy Positive,251,18
Actualy Negative,21,19


In [23]:
from matplotlib import pyplot as plt
tpr_list = []
fpr_list = []
#>> graf ROC curve

#>> pro kazdej threshold vypocitam confusion matrix, pak z toho nakreslim ROC curve

for t in range(1,100): # t= treshold
    tp = tn = fp = fn = 0
    threshold = t
    for p in predikce:
        if p > threshold:
            p = True
        else:
            p = False
        bool_predikce.append(p)
    #>> confusion matrix
    for i in range(0,len(bool_predikce)):
        if bool_predikce[i] and y[i]:
            tp += 1
        elif not bool_predikce[i] and not y[i]:
            tn += 1
        elif bool_predikce[i] and not y[i]:
            fp += 1
        elif not bool_predikce[i] and y[i]:
            fn += 1

        print(tp,fp,tn,fn)
        tpr = tp / (tp + fn) # true possitive rate
        fpr = fp / (fp + tn) # false possitive rate
        
        tpr_list.append(tpr)
        fpr_list.append(fpr)
        



plt.plot(fpr_list, tpr_list, marker='o', color='green')

1 0 0 0


ZeroDivisionError: division by zero

: 